# Decision Tree classifier

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Import files

In [6]:
File1 = 'iris.tmls'
File2 = 'wine.tmls'
File3 = 'car.tmls'

## Function which checks nominal features 

In [11]:
def _binary_coding(X3):
    _index = []
    l = 'n'
    for i in range(len( X3[0])):
            if X3[0, i:i+1] == l:
                _index.append(i)
    return _index 

# Data Processing

### Iris dataset

In [14]:
iris = pd.read_csv(File1)
#df1 = iris.drop(iris.index[0])
df1 = np.asarray(iris)
X1 , y1 = df1[:, :-1], df1[:, -1]
_index_feature1 = _binary_coding(X1)
print(_index_feature1)
df1 = pd.DataFrame(df1)
df1 = df1.drop(df1.index[0])
#dfs = df1.sample(frac=1)




#X1 = df1.iloc[:, :-1].values
#y1 = df1.iloc[:, -1].values.reshape(-1,1)
#X1 , y1 = df1[ :, :-1], df1[:, -1]
#print('X1', X1, 'y1', y1)


[]


### Wine dataset

In [15]:
wine = pd.read_csv(File2)
df2 = np.asarray(wine)
X2 , y2 = df2[:, :-1], df2[:, -1]
_index_feature2 = _binary_coding(X2)
print(_index_feature2)
df2 = pd.DataFrame(df2)
df2 = df2.drop(df2.index[0])

#print(df2)





#df2 = wine.drop(iris.index[0])
#df2 = df2.sample(frac=1)
#X2 , y2 = df2.iloc[:, :-1].values, df2.iloc[:, -1].values.reshape(-1,1)
#print('X2', X2, 'y2', y2)

[]


### Car dataset

In [16]:
car = pd.read_csv(File3)

df3 = np.asarray(car)
X3 , y3 = df3[:, :-1], df3[:, -1]
_index_feature3 = _binary_coding(X3)
print(_index_feature3)
df3 = pd.DataFrame(df3)
df3 = df3.drop(df3.index[0])



#print( 'length',len(df3.iloc[0, :]) - 1)
#print(len(_index_feature3))
#print('X3', X3, 'y3', y3)

[0, 1, 2, 3, 4, 5]


# Decision Tree Implementation in Python

In [17]:
class Node():
    def __init__(self, feature_index = None, threshold = None,left = None, right = None, info_gain= None, class_label=None):
        #Decision Node 
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.class_label = class_label
       
        

# Descision Tree model 

In [18]:
class descisionTree():
    
    def __init__(self, left_target=None, right_target = None, min_samples_split=2, max_depth=3):
        
        self.tree = None
      
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
    
    '''Tree builder'''
    def GrowTree(self, df, _index_feature, current_depth = 0):
        
        X, y = df[:, :-1], df[:, -1]
        self.D = df
        number_sample, list_feature = X.shape  
        #print('list_feature', list_feature, 'number_sample', number_sample)
        
        if number_sample >= self.min_samples_split and current_depth<=self.max_depth:
            
            best_split = self.BestSplit(df, number_sample, list_feature, _index_feature)
            
            if best_split["info_gain"]>0:
                left_tree = self.GrowTree(best_split['dataset_left'], _index_feature, current_depth+1)
                right_tree = self.GrowTree(best_split['dataset_right'], _index_feature, current_depth+1)
                return Node(best_split['feature_index'],best_split['threshold'], left_tree, right_tree, best_split['info_gain'])

        classes = self.calculate_label(y)
        #print('class_label', classes)
        return Node(class_label = classes)
       
    
    def BestSplit(self, dataset, number_sample, list_feature, _index_feature):
        
        best_split = {}
        
        max_info_gain = -1
        curr_info_gain = 0
        #_index_feature = self._index_feature            
        for feature_index in range(list_feature):
            feature_values = dataset[:, feature_index]
            #print('feature_values', feature_values)
            #unique_values = np.unique(feature_values)
            
            if feature_index not in _index_feature:
                thresholds = np.mean(feature_values.astype(dtype=np.float64))
                dataset[:, :-1] = dataset[:, :-1].astype(dtype=np.float64)
                dataset_left, dataset_right = self._split(dataset, feature_index, thresholds)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    #print('curr_info_gain', curr_info_gain)
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = thresholds
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
            else:
                thresholds = np.unique(feature_values) 
                #print('thresholds', thresholds)
                for th in thresholds:
                # get current split
                    #dataset = np.asarray(dataset)
                    dataset_left, dataset_right = self._split(dataset, feature_index, th)
                    # check if childs are not null
                    if len(dataset_left)>0 and len(dataset_right)>0:
                        y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                        # compute information gain
                        curr_info_gain = self.information_gain(y, left_y, right_y)
                        # update the best split if needed
                        if curr_info_gain>max_info_gain:
                            best_split["feature_index"] = feature_index
                            best_split["threshold"] = thresholds
                            best_split["dataset_left"] = dataset_left
                            best_split["dataset_right"] = dataset_right
                            best_split["info_gain"] = curr_info_gain
                            max_info_gain = curr_info_gain
        
        return best_split
            
    def _split(self, D , i , thresholds):
        
        data_left = np.array([row for row in D if row[i]< thresholds])
        data_right = np.array([row for row in D if row[i] >= thresholds])
        
        return data_left, data_right
    
    '''information gain from the split'''

    def information_gain(self, parent, l_child, r_child):
       
        #print('dataset', len(parent))
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        
        gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        
        return gain
    
    ''' compute gini index '''

    def gini_index(self, y):
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
     
    

    ''' Calculate class label for leaf node'''

    def calculate_label(self, y):
        
        y = list(y)
        return max(y, key=y.count)
   
 
    ''' Print the tree '''

    def print_tree(self, tree=None, indent=" "):
        
        if not tree:
            tree = self.tree

        if tree.class_label is not None:
            print(tree.class_label)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    def fit(self, X1, Y1, _index_feature):
        ''' function to train the tree '''
        
        df = np.concatenate((X1, Y1), axis=1)
        #print('df', df)
        self.tree = self.GrowTree(df, _index_feature)

    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.tree) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.class_label!= None: return tree.class_label
        feature_val = x[tree.feature_index]
        if (feature_val<=tree.threshold).all():
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
        



## Train the model 

In [19]:

def SplitTrainTest(df, split_ratio):
    df = np.asarray(df)
    index_split = int(len(df)*split_ratio)
    df_train = df[:index_split, :]
    df_test = df[index_split:, :]
    #print('df_train', df_train)
    #print('df_test',df_test)
    return  df_train[:, :-1], df_train[:, -1], df_test[:, : -1], df_test[:, -1 ] 

def Accuracy(Y_pred, y_test):
    numerator = 0
    for i in range(len(y_test)):
        if Y_pred[i] == y_test[i]:
            numerator +=1
    #print('numerator', numerator, 'denomerator', len(y_test))
    return numerator / len (y_test)

    
    
    


## Using built in library such as Scikit library to implement a Decision Tree Classifier

In [20]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



# function for performing decision tree classification using scikit library.

def SciDecisionTree(data):
    dataset = data
    for i in range(5):
        # Split the data into training and testing sets
    
        X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.3, random_state=42)
        
        clf = DecisionTreeClassifier(criterion='gini')

        # Train the classifier using the training data
        clf.fit(X_train, y_train)

        # Make predictions on the testing data
        y_pred = clf.predict(X_test)

        # Print the accuracy of the model
        print("Accuracy:", accuracy_score(y_test, y_pred))

## Test the model with three different dataset


### Decision Tree model to classify species Iris using the Iris dataset which contains features such as sepal length, sepal width, petal length, and petal width.

In [31]:


print('The Accuracy of a Decision Tree classifier for Iris Dataset.')

for i in range (5):
   
    df1 = df1.sample(frac=1)
    X_train, Y_train, X_test, Y_test = SplitTrainTest(df1, 0.8)
    y_train = list(Y_train)
    y_trains = [[row] for row in y_train]
    x_train = (X_train)
    y_test = [row for row in Y_test]
    model = descisionTree(min_samples_split=2, max_depth=3)
    model.fit(x_train, y_trains, _index_feature1)
    #model.print_tree()
    #print('X_train', x_train)
    #print('Y_train', y_trains)
    #print('X_test', X_test)
    #print('Y_test', Y_test)
    
    X_test = X_test.astype(dtype=np.float64)
    Y_pred = model.predict(X_test)
    accuracy = Accuracy(Y_pred, y_test)
    #print('Y_test', Y_test)
    print('Run',i,':', accuracy)
print('""""""""""""""""""""""""""""""')  

print('The Accuracy of a Decision Tree Classifier using scikit library for Iris Dataset')    
Iris = datasets.load_iris()
SciDecisionTree(Iris)
print('""""""""""""""""""""""""""""""')  
print('The Decision Tree looks like the following:')
model.print_tree()

The Accuracy of a Decision Tree classifier for Iris Dataset.
Run 0 : 1.0
Run 1 : 0.9333333333333333
Run 2 : 0.9333333333333333
Run 3 : 0.8333333333333334
Run 4 : 1.0
""""""""""""""""""""""""""""""
The Accuracy of a Decision Tree Classifier using scikit library for Iris Dataset
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
""""""""""""""""""""""""""""""
The Decision Tree looks like the following:
X_2 <= 3.7266666666666666 ? 0.27104648126387254
 left:X_2 <= 1.7260869565217392 ? 0.16162570888468802
  left:Iris-setosa
  right:X_2 <= 3.0 ? 0.375
    left:Iris-setosa
    right:Iris-versicolor
 right:X_3 <= 1.7054054054054055 ? 0.3551594121943883
  left:X_3 <= 1.4 ? 0.020833333333333315
    left:Iris-versicolor
    right:X_2 <= 4.7375 ? 0.06689814814814826
        left:Iris-versicolor
        right:Iris-versicolor
  right:X_1 <= 3.011764705882353 ? 0.0024715768660406173
    left:Iris-virginica
    right:X_3 <= 2.1928571428571426 ? 0.013605442176870833
        left:Iris

### Classifying Wines dataset based on their features using various attributes with the decision tree algorithm.

In [30]:
print('The Accuracy of a Decision Tree classifier for Wine Dataset.')
for i in range (5):
    df2 = df2.sample(frac=1)
    X_train, Y_train, X_test, Y_test = SplitTrainTest(df2, 0.8)
    y_train = list(Y_train)
    y_trains = [[row] for row in y_train]
    x_train = (X_train)
    y_test = [row for row in Y_test]
    model = descisionTree(min_samples_split=2, max_depth=3)
    model.fit(x_train, y_trains, _index_feature2)
    
    #model.print_tree()
    #print('X_train', x_train)
    #print('Y_train', y_trains)
    #print('X_test', X_test)
    #print('Y_test', Y_test)
    
    X_test = X_test.astype(dtype=np.float64)
    Y_pred = model.predict(X_test)
    accuracy = Accuracy(Y_pred, y_test)
    
    print('Run',i,':', accuracy)
print('""""""""""""""""""""""""""""""') 


print('The Accuracy of a Decision Tree Classifier from scikit library for Wine Dataset')
Wine = datasets.load_wine()
SciDecisionTree(Wine)

print('""""""""""""""""""""""""""""""')  
print('The Decision Tree looks like the following:')
#model.print_tree()

The Accuracy of a Decision Tree classifier for Wine Dataset.
Run 0 : 0.8888888888888888
Run 1 : 0.9166666666666666
Run 2 : 0.8611111111111112
Run 3 : 0.9166666666666666
Run 4 : 0.8055555555555556
""""""""""""""""""""""""""""""
The Accuracy of a Decision Tree Classifier from scikit library for Wine Dataset
Accuracy: 0.9629629629629629
Accuracy: 0.9629629629629629
Accuracy: 0.9629629629629629
Accuracy: 0.9629629629629629
Accuracy: 0.9629629629629629
""""""""""""""""""""""""""""""
The Decision Tree looks like the following:


### Decision Tree model used for analyzing and predicting car prices based on various features.

In [37]:
print('The Accuracy of a Decision Tree classifier for car Dataset with nominal feature value.')
df3 = np.asarray(df3)
l = len(df3[0, :])

df3 = pd.DataFrame(df3)
X3 = (df3.drop(df3.index[l-1], axis=1))

if len(_index_feature3) != 0:
    for j in range(len(_index_feature3)):
        unique_values = np.unique(X3.iloc[:, j: j+1]) 
        #print(unique_values)
        for i, value in enumerate(unique_values):
             X3.iloc[:, j: j+1] = np.where(X3.iloc[:, j: j+1] == value, i, X3.iloc[:, j: j+1])


df3.iloc[:, 0:l-1]= X3


for i in range (5):
    df3 = df3.sample(frac = 1)
    X_train, Y_train, X_test, Y_test = SplitTrainTest(df3, 0.8)
    y_train = list(Y_train)
    y_trains = [[row] for row in y_train]
    y_test = [row for row in Y_test]
    model = descisionTree(min_samples_split=2, max_depth=3)
    model.fit(X_train, y_trains, _index_feature3)
    X_test = X_test.astype(dtype=np.float64)
    Y_pred = model.predict(X_test)
    accuracy = Accuracy(Y_pred, y_test)
    
    print('Run',i,':', accuracy)
    
    
print('""""""""""""""""""""""""""""""')


print('The Accuracy of a Decision Tree Classifier from scikit library for Wine Dataset')
#Importing the car evaluation dataset from UCI repository
car_data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data', header=None)


for i in range(5):
        # Split the data into training and testing sets
        data = car_data.iloc[:, :-1]
        data = pd.get_dummies(data, prefix_sep='_')
        target = car_data.iloc[:, -1]
        X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=42)
        
        clf = DecisionTreeClassifier(criterion='gini')

        # Train the classifier using the training data
        clf.fit(X_train, y_train)

        # Make predictions on the testing data
        y_pred = clf.predict(X_test)

        # Print the accuracy of the model
        print("Accuracy:", accuracy_score(y_test, y_pred))

print('""""""""""""""""""""""""""""""')  
#print('The Decision Tree looks like the following:')
#model.print_tree()

The Accuracy of a Decision Tree classifier for car Dataset with nominal feature value.
Run 0 : 0.7630057803468208
Run 1 : 0.7427745664739884
Run 2 : 0.7947976878612717
Run 3 : 0.7658959537572254
Run 4 : 0.7947976878612717
""""""""""""""""""""""""""""""
The Accuracy of a Decision Tree Classifier from scikit library for Wine Dataset
Accuracy: 0.9518304431599229
Accuracy: 0.9595375722543352
Accuracy: 0.9576107899807321
Accuracy: 0.9614643545279383
Accuracy: 0.953757225433526
""""""""""""""""""""""""""""""
